In [ ]:
from threading import Thread
import random
from IPython.display import clear_output

# Função que calcula o produto escalar de dois vetores
def produtoEscalar(v1: list, v2: list) -> float:
    
    if len(v1) != len(v2):
        print("Não existe produto escalar de vetores de dimensões diferentes")
        return 0
    
    soma: float = 0
        
    for i in range(len(v1)):
        soma += v1[i] * v2[i]
    
    return soma

# Função que cria um vetor de dimensão N com números aleatórios entre [-intervalo, intervalo]
def pegarAleatorio(intervalo: int, n: int) -> list:
    v: list = []
    for i in range(n):
        v.append(random.randint(-intervalo, intervalo))
    return v

# Resposta dada pelo usuário
resposta: float = 0
    
# Flag: Se o timer chegou a zero
acabouTempo: bool = False

# Flag: Se foi recebido um input (i.e, a função perguntarResposta() terminou)
recebeuInput: bool = False

# Flag: Usado para rodar o jogo uma vez antes de perguntar se quer jogar denovo
temp: bool = False

# Variáveis de jogo
timeout: int = 10  # Tempo de cada rodada
vezesJogadas: int = 0
vezesGanhas: int = 0

dificuldadeAtual: int = 0

# Usado para ir mudando as dificuldades conforme acertos
count: int = 25

# countDiffs = [25, 21, 18, 15, 12, 9, 6, 3, 0]
    
# Nomes das dificuldades    
nomeDiffs: list = [
    "Estudante de humanas",  # 25
    "Nutella",  # 21
    "Fácil",  # 18
    "Médio",  # 15
    "Difícil",  # 12
    "Bem difícil",  # 9
    "Fodão",  # 6
    "Calculadora humana",  # 3
    "'Vocês viram isso no ensino médio' -Elvio"  # 0
]
    
# Tempo para cada dificuldade
tempoDiffs: list = [20, 20, 20, 20, 25, 30, 25, 20, 15]

# Dimensão para cada dificuldade
dimDiffs: list = [1, 1, 2, 2, 3, 3, 4, 5, 6]

# Intervalo para cada dificuldade
ranDiffs: list = [3, 4, 5, 7, 10, 12, 15, 20, 20]


def perguntarResposta():
    global recebeuInput
    
    resposta = float(input("Insira o produto escalar: "))
    recebeuInput = True
    return resposta


# Função a ser rodada em thread que faz a contagem
def iniciarContagem(tempo: int):
    global recebeuInput
    global acabouTempo

    print("Iniciando contagem! ")
    
    while (tempo > 0):
        # Se recebeu input, para de esperar
        if (recebeuInput == True):
            print("\nRecebido input\n")
            break
        
        # Printa o tempo restante, espera 1s e diminui 1s do tempo restante
        print(f"{tempo}s", end=" ")
        time.sleep(1)
        tempo -= 1
    
    if (not recebeuInput):
        print("\nAcabou o tempo!")
        print("Insira qualquer valor para continuar") 
        # O usuário não colocou nenhum input a tempo
        acabouTempo = True
    
    # Sai da thread
    return
    

print("Escolha sua dificuldade\n")
for i, nome in enumerate(nomeDiffs):
    print(f">> {i} {nome}")

try:
    dificuldadeAtual = int(input(">> "))
    print(dificuldadeAtual)
    if dificuldadeAtual > len(nomeDiffs) - 1:
        raise ValueError
except ValueError:
    dificuldadeAtual = 0
    
timeout = tempoDiffs[dificuldadeAtual]
count = countDiffs[dificuldadeAtual]    

clear_output(wait=False)
    
# Loop principal
while True:
    

    if count > 21:
        dificuldadeAtual = 0
    elif count > 18:
        dificuldadeAtual = 1
    elif count > 15:
        dificuldadeAtual = 2
    elif count > 12:
        dificuldadeAtual = 3
    elif count > 9:
        dificuldadeAtual = 4
    elif count > 6:
        dificuldadeAtual = 5
    elif count > 3:
        dificuldadeAtual = 6
    elif count > 0:
        dificuldadeAtual = 7
    else:
        dificuldadeAtual = 8
        
    print(f"Dificuldade atual: {nomeDiffs[dificuldadeAtual]}")
    
    # Se não é a primeira vez jogando
    if (temp):
        try:
            i = int(input("Escolha uma opção:\n[0] >> Sair\n[1] >> Jogar novamente\n"))
            if (i == 0):
                print("Até a próxima")
                print(f"Você ganhou {vezesGanhas} partidas de {vezesJogadas}")
                break
        except ValueError:
                pass
    
        clear_output(wait=False)
    
    u = pegarAleatorio(ranDiffs[dificuldadeAtual], dimDiffs[dificuldadeAtual])
    v = pegarAleatorio(ranDiffs[dificuldadeAtual], dimDiffs[dificuldadeAtual])
    
    res = produtoEscalar(u, v)
    
    print(f"Calcule o produto escalar de: \nU = {u}\nV = {v}")
    
    # Thread alternativa para fazer a contagem
    # Thread principal ficará para pegar o input
    t1 = Thread(target=iniciarContagem, args=[timeout])
    t1.start()
    resposta = perguntarResposta()
    
    # Faz o sistema esperar a thread terminar antes de prosseguir no loop
    t1.join()
    if (resposta == res and not acabouTempo):
        print("Parabéns, tu é brabo mesmo")
        count -= 1
        vezesGanhas += 1
    elif (acabouTempo):
        print("Demorou demais")
        count += 1
    else:
        print("Resposta errada")
        count += 1
    
    print(f"A resposta era {res}")
    
    recebeuInput = False
    acabouTempo = False
    temp = True
    
    vezesJogadas += 1